In [41]:
import numpy as np

In [48]:
bbox_3d_sample = np.array([
    [
        [ 5.0623603, 0.502089,    8.64212   ],
        [ 3.447396,    0.5278923,   8.750486  ],
        [ 3.470547,    1.8430142,   8.782355  ],
        [ 5.085511,    1.8172109,   8.673988  ],
        [ 5.326024,    0.40165097, 12.595361  ],
        [ 3.7110596,   0.42745423, 12.703728  ],
        [ 3.7342103,   1.7425761,  12.735596  ],
        [ 5.3491745,   1.7167728,  12.62723   ]
    ]
])
print(bbox_3d_sample.shape)

bbox_3d_sample = np.load("sample_bbox.npy")
bbox_class = np.load("sample_class.npy")
print(bbox_3d_sample.shape)
print(bbox_3d_sample)
print(bbox_class)

(1, 8, 3)
(8, 8, 3)
[[[-8.6518421e+00  4.2267144e-02  2.0507565e+01]
  [-9.2138643e+00  4.2400837e-02  2.0259998e+01]
  [-9.1981773e+00  1.8229675e+00  2.0225346e+01]
  [-8.6361551e+00  1.8228338e+00  2.0472914e+01]
  [-9.0261002e+00  6.2099457e-02  2.1357212e+01]
  [-9.5881224e+00  6.2233150e-02  2.1109644e+01]
  [-9.5724354e+00  1.8427999e+00  2.1074993e+01]
  [-9.0104132e+00  1.8426661e+00  2.1322559e+01]]

 [[-9.3636084e+00 -1.1039066e-01  3.6413883e+01]
  [-8.6891737e+00 -1.0742706e-01  3.6462559e+01]
  [-8.6974154e+00  1.6874189e+00  3.6467484e+01]
  [-9.3718500e+00  1.6844553e+00  3.6418804e+01]
  [-9.3054800e+00 -1.0791427e-01  3.5608353e+01]
  [-8.6310453e+00 -1.0495067e-01  3.5657032e+01]
  [-8.6392870e+00  1.6898953e+00  3.5661953e+01]
  [-9.3137217e+00  1.6869316e+00  3.5613277e+01]]

 [[ 6.8332143e+00 -2.5658011e-02  2.0014341e+01]
  [ 8.5988207e+00  1.4121413e-02  1.9635206e+01]
  [ 8.5529366e+00  1.9061353e+00  1.9620039e+01]
  [ 6.7873302e+00  1.8663559e+00  1.9999174e+

In [58]:
def output2MarkerArray(bboxes_per_image, classes_per_image):
    def get_quaternion_from_euler(roll, pitch, yaw):
        """
        Convert an Euler angle to a quaternion.

        Input
            :param roll: The roll (rotation around x-axis) angle in radians.
            :param pitch: The pitch (rotation around y-axis) angle in radians.
            :param yaw: The yaw (rotation around z-axis) angle in radians.

        Output
            :return qx, qy, qz, qw: The orientation in quaternion [x,y,z,w] format
        """
        qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
        qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
        qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
        qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
        
        return [qx, qy, qz, qw]

    marker_list = []

    for single_bbox, single_class in zip(bboxes_per_image, classes_per_image):
        # BBOX is stored as (8, 3) -> where last_index is (y, z, x)
        min_x, max_x = np.min(single_bbox[:, 2]), np.max(single_bbox[:, 2])
        min_y, max_y = np.min(single_bbox[:, 0]), np.max(single_bbox[:, 0])
        min_z, max_z = np.min(single_bbox[:, 1]), np.max(single_bbox[:, 1])

        # Center points of BBOX along each axis
        cx = (min_x + max_x) / 2
        cy = (min_y + max_y) / 2
        cz = (min_z + max_z) / 2

        # Size of BBOX along each axis
        scale_x = max_x - min_x
        scale_y = max_y - min_y
        scale_z = max_z - min_z

        # Rotation of BBOX along each axis
        yaw_angle = np.math.atan2((single_bbox[1, 2] - single_bbox[0, 2]), (single_bbox[1, 0] - single_bbox[0, 0]))
        roll_angle = np.math.atan2((single_bbox[1, 1] - single_bbox[0, 1]), (single_bbox[1, 0] - single_bbox[0, 0]))
        pitch_angle = np.math.atan2((single_bbox[4, 2] - single_bbox[0, 2]), (single_bbox[4, 1] - single_bbox[0, 1])) + 1.578
        
        # TODO: Right now appending values to a list, later insert values in right places in marker array. DON'T forget class
        marker_list.append((cx, cy, cz, scale_x, scale_y, scale_z, get_quaternion_from_euler(roll_angle, pitch_angle, yaw_angle)))

    return marker_list

print(output2MarkerArray(bbox_3d_sample, bbox_class))

[(20.791278839111328, -9.112138748168945, 0.9425334930419922, 1.1318665, 0.95196724, 1.8005328, [0.0017779533402660125, -0.007869390610779814, -0.20596873036779825, -0.9785253158115083]), (36.03791809082031, -9.001447677612305, 0.7897523045539856, 0.85913086, 0.7408047, 1.8002859, [0.0020105481234790304, 0.005214711675525441, 0.03600407953118648, 0.9993360149296614]), (17.72915267944336, 7.245325565338135, 0.9126417636871338, 4.5703773, 2.7069902, 1.9869871, [0.010882647417044542, -0.00417807192823888, -0.10552289594164317, 0.9943485455976933]), (33.01568603515625, -9.965767860412598, 0.7994295358657837, 0.65514374, 0.69428635, 1.7283046, [-0.0030297020242300203, 0.006305835736213831, -0.04823271128310024, 0.998811625334624]), (24.811628341674805, 5.073976993560791, -0.13186508417129517, 9.517551, 5.398589, 3.800397, [-0.00205499625340626, 0.016930544782572925, 0.1652093150417087, 0.986111056558648]), (45.395545959472656, -1.8215103149414062, 0.6504349708557129, 4.723297, 1.8603, 2.366

In [59]:
np.load("dd3d/outputs/marker_list_30.npy")

array([[ 2.05090733e+01, -9.95841980e+00,  8.46440673e-01,
         8.68415833e-01,  8.36074829e-01,  1.74834371e+00,
         1.50392303e-03, -6.57852941e-04,  2.17984140e-01,
         9.75950931e-01],
       [ 2.35445061e+01, -8.88214397e+00,  8.17135811e-01,
         8.61930847e-01,  6.93838120e-01,  1.82311964e+00,
        -1.66872400e-03,  3.09105361e-03,  5.78116768e-02,
        -9.98321326e-01],
       [ 4.05518417e+01, -1.90501511e+00,  7.68482745e-01,
         5.18816376e+00,  5.71545219e+00,  2.46404839e+00,
         3.28501075e-03,  9.45492173e-03,  4.33119273e-01,
         9.01281037e-01],
       [ 4.42827339e+01, -8.61059666e+00,  1.23966897e+00,
         1.77883148e+00,  4.03394127e+00,  1.51126456e+00,
         1.92731155e-01,  2.26610960e-01,  6.56320105e-01,
         6.93358561e-01],
       [ 1.62522182e+01,  5.73645258e+00, -3.24742794e-02,
         8.58908367e+00,  4.78267384e+00,  3.79635859e+00,
        -4.79930443e-03,  1.88197015e-02,  1.51256304e-01,
         9.